# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

### De clicks, me quedo con sólo datos que correspondan a la competencia

In [4]:
clicks_competencia = clicks.loc[clicks["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [5]:
# Ventanas de tres dias
Xs = {}
try:
    print("Cargando features")
    for ventana_nro in range(1,7):    
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_clicks.csv".format(ventana_nro), index_col=0)
except FileNotFoundError:
    print("No se encontraron los features, se generarán los dataframe requeridos")
    for ventana_nro in range(1,7):    
        Xs[ventana_nro] = clicks.loc[clicks["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
No se encontraron los features, se generarán los dataframe requeridos


## Set que se usará para predecir

In [6]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_clicks.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


## Diccionario de features

In [7]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[1].columns.values)

---

# Creación de features

## Estadísticas sobre el tiempo hasta clickear

In [8]:
def tiempo_hasta_clickear_media(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"timeToClick" : "mean"}).rename(columns={"timeToClick": "tiempo_hasta_click_mean_clicks"})

In [9]:
features["tiempo_hasta_clickear_media"] = (tiempo_hasta_clickear_media, ["tiempo_hasta_click_mean_clicks"])

In [10]:
def tiempo_hasta_clickear_max(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"timeToClick" : "max"}).rename(columns={"timeToClick": "tiempo_hasta_click_max_clicks"})

In [11]:
features["tiempo_hasta_clickear_max"] = (tiempo_hasta_clickear_max, ["tiempo_hasta_click_max_clicks"])

In [12]:
def tiempo_hasta_clickear_min(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"timeToClick" : "min"}).rename(columns={"timeToClick": "tiempo_hasta_click_min_clicks"})

In [13]:
features["tiempo_hasta_clickear_min"] = (tiempo_hasta_clickear_min, ["tiempo_hasta_click_min_clicks"])

In [14]:
def tiempo_hasta_clickear_std(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"timeToClick" : "std"}).rename(columns={"timeToClick": "tiempo_hasta_click_std_clicks"}).fillna(0)

In [15]:
features["tiempo_hasta_clickear_std"] = (tiempo_hasta_clickear_std, ["tiempo_hasta_click_std_clicks"])

---

## Cantidad de clicks dentro de la ventana

In [16]:
def cantidad_de_clicks(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"timeToClick" : "count"}).rename(columns={"timeToClick": "cantidad_clicks_clicks"})

In [17]:
features["cantidad_de_clicks"] = (cantidad_de_clicks, ["cantidad_clicks_clicks"])

---

## Cantidad de clicks por período horario

In [18]:
def cantidad_apariciones_por_periodo_horario(dataframe, nro_ventana):
    features = dataframe.groupby("device_id").agg( \
       {"hora_2_a_6" : "sum",
        "hora_6_a_11" : "sum",
        "hora_11_a_14" : "sum",
        "hora_14_a_18" : "sum",
        "hora_18_a_23" : "sum",
        "hora_23_a_2" : "sum"})
    features.columns = ["apariciones_" + x + "_clicks" for x in features.columns]
    return features

In [19]:
columnas_generadas = ["apariciones_" + x + "_clicks" for x in ["hora_2_a_6", "hora_6_a_11", "hora_11_a_14", "hora_14_a_18", "hora_18_a_23", "hora_23_a_2"]]
features["cantidad_apariciones_por_periodo_horario"] = (cantidad_apariciones_por_periodo_horario, columnas_generadas)

---

## Tiempo medio entre clicks

In [20]:
def tiempo_medio_entre_clicks(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).mean().total_seconds()))}).rename(columns={"date": "tiempo_medio_entre_clicks"})

In [21]:
features["tiempo_medio_entre_clicks"] = (tiempo_medio_entre_clicks, ["tiempo_medio_entre_clicks"])

---

## Tiempo máximo entre clicks

In [22]:
def tiempo_maximo_entre_clicks(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).max().total_seconds()))}).rename(columns={"date": "tiempo_maximo_entre_clicks"})

In [23]:
features["tiempo_maximo_entre_clicks"] = (tiempo_maximo_entre_clicks, ["tiempo_maximo_entre_clicks"])

---

## Tiempo mínimo entre clicks

In [24]:
def tiempo_minimo_entre_clicks(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).min().total_seconds()))}).rename(columns={"date": "tiempo_minimo_entre_clicks"})

In [25]:
features["tiempo_minimo_entre_clicks"] = (tiempo_minimo_entre_clicks, ["tiempo_minimo_entre_clicks"])

---

## Desvío entre clicks

In [26]:
def desvio_entre_clicks(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).std().total_seconds()))}).rename(columns={"date": "desvio_entre_clicks"})

In [27]:
features["desvio_entre_clicks"] = (desvio_entre_clicks, ["desvio_entre_clicks"])

---

## Suma tiempos entre clicks

In [28]:
def tiempo_entre_apariciones_suma(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : (lambda x: (x.sub(x.shift()).sum().total_seconds()))}).rename(columns={"date": "tiempo_entre_clicks_suma_clicks"})

In [29]:
features["tiempo_entre_clicks_suma"] = (tiempo_entre_apariciones_suma, ["tiempo_entre_clicks_suma_clicks"])

---

# Genero los features

## Función generadora de features en ventanas

In [30]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")

### Genero los features

In [31]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(clicks, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7)
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(clicks_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8)
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

Generando feature tiempo_hasta_clickear_media en ventanas
Generando feature tiempo_hasta_clickear_media en set de predicciones
Generando feature tiempo_hasta_clickear_max en ventanas
Generando feature tiempo_hasta_clickear_max en set de predicciones
Generando feature tiempo_hasta_clickear_min en ventanas
Generando feature tiempo_hasta_clickear_min en set de predicciones
Generando feature tiempo_hasta_clickear_std en ventanas
Generando feature tiempo_hasta_clickear_std en set de predicciones
Generando feature cantidad_de_clicks en ventanas
Generando feature cantidad_de_clicks en set de predicciones
Generando feature cantidad_apariciones_por_periodo_horario en ventanas
Generando feature cantidad_apariciones_por_periodo_horario en set de predicciones
Generando feature tiempo_medio_entre_clicks en ventanas
Generando feature tiempo_medio_entre_clicks en set de predicciones
Generando feature tiempo_maximo_entre_clicks en ventanas
Generando feature tiempo_maximo_entre_clicks en set de predicc

---

# Guardado de los sets de entrenamiento y el set de prediccion

In [33]:
for ventana_nro in range(1, 7):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_clicks.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_clicks.csv")

---